# Matrizes e vetores em Julia

  - [Criando e acessando vetores e matrizes](#Criando-e-acessando)
  - [Percorrendo com a ajuda de laços](#Percorrendo-com-laços)
  - [Uso de memória](#Memória)

## Criando e acessando

Para criar vetores para números reais, o comando genérico é

    Vector{Float64}(tamanho)
    
porém, existem outros comandos para criar vetores especiais: `ones`, `zeros` e `rand`.

Ao criar um vetor com o comando `Vector` recebemos apenas um espaço na memória, cheio de sujeira.

In [ ]:
v = Vector{Float64}(undef, 5)

In [ ]:
pointer(v) # Esse e' o endereco do vetor

In [ ]:
sizeof(v) # Tamanho em bytes ocupado pelo vetor

O comando abaixo cria um vetor de zeros. Como estamos guardando em `v` novamente, perdemos o endereço criado acima.

In [ ]:
v = zeros(5)

In [ ]:
pointer(v)

In [ ]:
ones(5)

In [ ]:
rand(5)

Para trabalhar com vetores (e matrizes também) utilizamos índices dentro de colchetes. A seguir criamos um vetor de zeros, colocamos o valor 1 na terceira posição e guardamos na quinta posição o resultado da soma de seus valores na primeira e terceira posições.

In [ ]:
v = zeros(7)

# Armazena o valor 1 na posicao 3
v[3] = 1.0

# Armazena na posicao 5 o resultado da soma dos valores contidos nas posicoes 1 e 3
v[5] = v[1] + v[3]

println(v)

Para criar matrizes de números reais, o comando

    Array(Float64, m, n)
    
reserva um espaço para uma matriz com $m$ linhas e $n$ colunas e devolve o seu endereço na memória. Dentro dessa matriz não sabemos o que tem.

De forma análoga, temos os comandos `zeros`, `ones` e `rand` para matrizes, que criam e colocam valores na matriz criada.

In [ ]:
M = Array{Float64}(undef, 5, 3) # Pode ter qualquer coisa nas posicoes da matriz criada dessa forma

In [ ]:
pointer(M)

In [ ]:
sizeof(M)

In [ ]:
M = zeros(5, 3)

In [ ]:
M = ones(5, 3)

In [ ]:
M = rand(5, 3)

A posição $(i, j)$ de uma matriz $M$ é acessada também utilizando colchetes, mas agora com duas dimensões, separadas por vírgula.

In [ ]:
# Cria uma matriz de 1s
M = ones(2, 7)

# Coloca os valor 10 na posição (1,5), substituindo o valor existente
M[1, 5] = 10.0

# Imprime o resultado da soma dos elementos (1,5), (2,1) e (2,7)
soma = M[1, 5] + M[2, 1] + M[2, 7]

println("O valor da soma e' $(soma).")

Colunas e linhas de uma matriz podem ser acessadas usando `:`, que significa **todas**. Por exemplo,

    M[:, 2]
    
seleciona **todas as linhas** da coluna 2 de $M$. Por outro lado

    M[2, :]
    
seleciona **todas as colunas** da linha 2 de $M$.

Nos comandos abaixo, selecionamos $M^2$ e $M_2$.

In [ ]:
M = rand(2, 4)
display(M)

println(M[:, 2])

println("Dimensões da coluna: $(size(M[:, 2]))\n")

println(M[2, :])

println("Dimensões da linha: $(size(M[2, :]))\n")

O comando `size` devolve as dimensões $m$, e $n$ da matriz. No caso acima, ambos são vetores com tamanhos 2 e 4, respectivamente.

**Cuidado!** O comando abaixo cria uma *cópia* da coluna de $M$, não é como se ele fosse o endereço dessa parte da matriz. No exemplo abaixo, uma modificação em `coluna` não causa uma modificação na matriz `M` (a recíproca também é verdadeira).

In [ ]:
M = zeros(3, 3)

# Coluna e um vetor (coluna) com 3 elementos
coluna = M[:, 2]

coluna[1] = 2.0

display(coluna)

display(M)

M[3, 2] = - 1.0

display(M)

display(coluna)

## Percorrendo com laços

Geralmente usamos laços do tipo `for` para percorrer vetores e matrizes. A razão para isso é que geralmente sabemos seu tamanho e temos que percorrê-los por inteiro.

Os comandos abaixo *inicializam* um vetor $v$ de forma que $v_i = i^2$.

In [ ]:
v = Vector{Float64}(undef, 10)

for i = 1:length(v)
    v[i] = i^2
end

println(v)

O comando `length` devolve o tamanho de um vetor.

Abaixo, somamos todas as posições do vetor $v$ criado acima: $\sum \limits_{i = 1}^n v_i$

In [ ]:
soma = 0.0
for i = 1:length(v)
    soma = soma + v[i]
end

println("A soma e' $(soma).")

Para matrizes, necessitamos de 2 laços: um para linhas e outro para colunas

In [ ]:
M = Array{Float64}(undef, 5, 10)

m, n = size(M)

# Em Julia, e' mais inteligente fixar a coluna primeiro
for j = 1:n
    for i = 1:m
        M[i, j] = i + j
    end
end

display(M)

Abaixo, somamos todas as posições do vetor $M$ criado acima: $\sum \limits_{j = 1}^n \sum \limits_{i = 1}^m m_{ij}$

In [ ]:
s = 0.0

for j = 1:n
    for i = 1:m
        s = s + M[i, j]
    end
end

println("A soma e' $(s)")

Percorrendo a diagonal de uma matriz quadrada

In [ ]:
M = rand(5, 5)

for i = 1:5
    println(M[i, i])
end

## Memória

### Acesso

A forma como acessamos matrizes e vetores na memória tem uma grande influência na eficiência do código computacional. Tudo é influenciado pela forma como a linguagem de programação armazena a matriz na memória. Além disso, existe uma memória adicional, chamada *cache* que armazena temporariamente pequenas quantidades do vetor.

Vamos ver o impacto de acessar elementos distantes na memoria em um vetor. Para isso criamos as funções abaixo, que realizam a soma dos elementos de um vetor `v`.

In [ ]:
"""
Esta funcao realiza a soma dos elementos do vetor `v` de forma ineficiente
"""
function soma_lento(v)
    
    n = length(v)
    
    n2 = Int(round(n / 2))
    
    s = 0.0
   
    # Esta soma olha para 2 posicoes distantes na memoria
    for i = 1:n2
        
        s = s + v[2 * i - 1]
        
    end
    for i = 1:n2
        
        s = s + v[2 * i]
        
    end
    
    return s
    
end

"""
Esta funcao realiza a soma dos elementos do vetor `v` aproveitando o conhecimento de que
posicoes proximas sao acessadas de forma mais rapida.
"""
function soma_rapido(v)
    
    n = length(v)
    
    n2 = Int(round(n / 2))
    
    s = 0.0
   
    # Soma dois valores proximos na memoria
    for i = 1:n2
        
        s = s + v[i]
        
    end
    for i = n2 + 1:n
        
        s = s + v[i]
        
    end
    
    return s
    
end

Para medir o tempo, primeiramente temos que chamar as funções uma vez, com um vetor pequeno, apenas para o Julia compilar eficientemente o código.

In [ ]:
vetor = ones(10); # cria um vetor de 1's.

soma_lento(vetor);

soma_rapido(vetor);

Agora criamos um vetor aleatório com tamanho grande e medimos o tempo que cada função leva. **Atenção** ao criar um vetor muito grande, pois sua máquina pode travar. Quanto maior o vetor, maior a diferença entre os tempos.

In [ ]:
# Cria um vetor aleatorio. O tamanho escolhido deve ser natural e par.
# Experimente aumentar o tamanho aos pouquinhos e ver como a diferença aumenta.
vetor = rand(100000000)

# Calcula o tempo para a soma ineficiente
println("Lento: ", @elapsed(soma_lento(vetor)))

# Calcula o tempo para a soma mais eficiente
println("Rapido: ", @elapsed(soma_rapido(vetor)))

O comando `@elapsed` conta o tempo para executar o comando passado por argumento.

## Matrizes na memória

Agora que observamos que valores próximos de um elemento do vetor são acessados mais rapidamente do que valores mais distantes, podemos estender o raciocínio para matrizes.

Seja $M$ uma matriz $m \times n$ e considere o elemento $M_{i,j}$. Qual elemento você acha que está mais próximo de $M_{i,j}$?

  - $M_{i + 1, j}$
  - $M_{i, j + 1}$
  - $M_{i + 1, j + 1}$
  - Todos
  
A resposta é: depende da linguagem de programação.

Em Julia, matrizes são criadas na memória por colunas (*column oriented*, em inglês). Na linguagem de programação `C`, por exemplo, elas são criadas por linhas (*row oriented*). `Fortran` também é em colunas.

In [ ]:
M = Array{Float64}(undef, 10, 10)

O Código abaixo, simplesmente coloca números de 1 a 100 na matriz `M` criada acima. Como veremos adiante, a forma abaixo é a forma incorreta de acessar `M` em Julia.

In [ ]:
k = 1.0

for i = 1:10
    
    for j = 1:10
        
        M[i, j] = k
        
        k = k + 1
        
    end
    
end

display(M)

Para verificarmos inicialmente que matrizes são armazenadas por colunas, basta notarmos que Julia permite que acessemos os elementos de uma matriz como se ela fosse um vetor. O que você acha que é o elemento `M[15]`?

In [ ]:
M[15]

Observe que interessante o que ocorre quando percorremos a *matriz* `M` como um *vetor* em Julia!

In [ ]:
for i = 1:100
    println(M[i])
end

Podemo ver que o elemento $M_{10,1}$ está mais próximo, *na memória*, de $M_{1,2}$ que de $M_{10, 2}$!

In [ ]:
function inicializa_por_linha(M)
    
    k = 0.0
    
    m, n = size(M)
    
    # Fixa a linha
    for i = 1:m
        
        # Percorre as colunas
        for j = 1:n
            
            M[i, j] = k
            
            k = k + 1
            
        end
        
    end
    
end

function inicializa_por_coluna(M)
    
    k = 0.0
    
    m, n = size(M)
    
    # Fixa uma coluna
    for j = 1:n
        
        # Percorre as linhas
        for i = 1:m
            
            M[i, j] = k
            
            k = k + 1
            
        end
        
    end
       
end

In [ ]:
M = zeros(10, 10);

@elapsed(inicializa_por_linha(M));

@elapsed(inicializa_por_coluna(M));

In [ ]:
m = 1000
n = 1000

M = Array{Float64}(undef, m, n)

println("Por linha: ", @elapsed(inicializa_por_linha(M)))

println("Por coluna:", @elapsed(inicializa_por_coluna(M)))

**Observação**. Note que cada função gera uma matriz diferente (teste com $n$ e $m$ iguais a 10 para ver). Um exercício interessante é saber como modificar o código de colunas para produzir a mesma matriz da função por linhas (ou vice-versa).

### Economizando memória

Você sabe a diferença entre os dois blocos de comando abaixo?

In [ ]:
v1 = zeros(100)
#println(pointer(v1))
v1[10] = -10.0

v1 = zeros(100);
#println(pointer(v1))

In [ ]:
v2 = Vector{Float64}(undef, 100)

for i = 1:length(v2)
    v2[i] = 0.0
end
#println(pointer(v2))

v2[10] = -10.0

for i = 1:length(v2)
    v2[i] = 0.0
end
#println(pointer(v2))

Remova os comandos comentados e rode-os novamente. Observe que o endereço da memória no primeiro bloco é alterado, enquanto que no segundo não. Isso significa que toda vez que o comando `zeros` é chamado, ele cria um **novo** vetor, reservando um **novo** espaço na memória!

In [ ]:
function zera_muita_memoria()
    
    for k = 1:1000
        
        v = zeros(100)
        
        v[1] = 10.0
    end
    
end

function zera_pouca_memoria()

    v = Vector{Float64}(undef,100)
    
    for k = 1:1000
        
        for i = 1:length(v)
            v[i] = 0.0
        end
        
        v[1] = 10.0
    end
    
end    

In [ ]:
@time zera_muita_memoria()

In [ ]:
@time zera_pouca_memoria()

O comando `@time` devolve o <U>tempo gasto</U> e o <U>número de alocações</U> de espaço na memória realizadas pela função.

A primeira função utiliza aproximadamente **800** vezes mais memória que a segunda! Note que, ao invés de `Vector` poderíamos ter utilizado `zeros` na segunda função, contanto que estivesse *fora* do laço em $k$.

Ir para [Parte 2](parte2.ipynb)